# Tapering off qubits using spatial symmetries in $BeH_2$


In [4]:
# importing stuff
import logging
import copy
import itertools
# with warnings.catch_warnings():
#     warnings.simplefilter("ignore", category=PendingDeprecationWarning)
#     from third_party_lib import some_module
from qiskit.quantum_info import Pauli
from pyscf import gto, scf, ao2mo
from pyscf.lib import param
from scipy import linalg as scila
from pyscf.lib import logger as pylogger
# from qiskit.chemistry import AquaChemistryError
from qiskit.chemistry import QMolecule
import numpy as np
from qiskit.aqua import Operator
from qiskit.aqua.algorithms import ExactEigensolver
# import gse_algo as ga
import scipy
from pyscf.scf.hf import get_ovlp
from symmetries import find_symmetry_ops
from qiskit.chemistry import FermionicOperator
from int_func import qmol_func

logger = logging.getLogger(__name__)
import int_func
from int_func import qmol_func
from taper_qubits_rm_funcs import *
# import taper_qubits_rm_funcs.qubit_tapering
# from taper_qubits_rm_funcs import r_mat_func
# from taper_qubits_rm_funcs import qubit_tapering
from r_mat_for_mols import mol_r_matrices, check_commute
import warnings
warnings.simplefilter("ignore", category=PendingDeprecationWarning)


ModuleNotFoundError: No module named 'qiskit.aqua.operators'

In [14]:
# Change the following flag to true to print the R matrices.
FLAG_PRINT_R_MATRICES = False
check_ref_energy = False
AO = True
MF = 'BeH2'
check_r_mat_commut=True
run_vqe = False
# This calculates the reference eigenspectrum. Make the following
# flag true, only when the system is small.
check_ref_energy = False
# r_mat_func 
x = r_mat_funcs(MF, check_r_mat_commut,AO)

# printing the R-matrices for BeH2
counter = 1
if FLAG_PRINT_R_MATRICES:
    for i in x.r_matrices:
        print('R'+str(counter)+' = ')
        print(i)
        counter+=1
# The R-matrices are simultaneously diagonalized
[r_mat_evals,v_matrix] = x.sim_diag(x.r_matrices)


BeH2
converged SCF energy = -15.5613526278409
True
True
True
True
All the above matrices work!
checking the v matrix...
v matrix is  unitary.


In [16]:
# Getting the qubit opertor form of Hamiltonian
qub_op = x.fer_op.mapping('jordan_wigner')
# Total number of terms in the qubit operator Hamiltonian
qub_op.chop()
print('Number of terms in the Hamiltonian in AO basis')
print(len(qub_op._paulis))

# Getting and transforming the Ham with V matrix.
v_qubit_op = x.sym_transf_ham_qub_op(v_matrix)

#Printing the number of terms in the Hamiltonian
print('Final number of terms in the Hamiltonian in AO basis after the transformation')
print(len(v_qubit_op._paulis))

Number of terms in the Hamiltonian in AO basis
1424
Final number of terms in the Hamiltonian in AO basis after the transformation
824


In [18]:
# Sanity check. Calculate the spectrum of Hamiltonian

if check_ref_energy ==True:
    ee = ExactEigensolver(qub_op, k=1)
    ee_result = ee.run()
#   Get the first 6 eigenvalues of the Hamiltonian spectrum
    ref_min_eigvals = ee_result['eigvals'][0:6]
    # This is the reference value from Hamiltonian in AO basis
    print('Eigen value of the full Ham in AO basis')
    print(ref_min_eigvals)

# The set of symmetries is not independent, so, the following code gets the independent set of symmetries.
r_mat_evals = x.ind_symm_r_ev_mat(r_mat_evals)
sym_list = x.get_symm_list(r_mat_evals)



/Users/kanavsetia/q_code/test_repos/qiskit-aqua/qiskit/aqua/operator.py:1691: DeprecationWarning: The `Operator` class is deprecated and will be removed after 0.6. Use the class for each representation instead, including `MatrixOperator`, `WeightedPauliOperator` and `TPBGroupedWeightedPauliOperator`
  DeprecationWarning)


In [ ]:
# In order to check if the symmetries commute with Ham, uncomment the following piece of code:
print("check the commutativity of the found symmetry paulis between H'.")
for symm in sym_list:
    symm_op = Operator(paulis=[[1.0, symm]])
    is_commutes = check_commute(symm_op, v_qubit_op)
    print(symm_op.print_operators())
    # symm_op.to_matrix()
    # print('Trace of the operators')
    # print(np.trace(symm_op._matrix.todense()))
    sym_la = symm.to_label()[::-1]
    ind = [i for i, a in enumerate(sym_la) if a == 'Z']
    print(ind)
    print("symm is {} commutes.".format("" if is_commutes else "NOT"))

# exit()
# Get the unitary operators (cliffords) corresponding the single qubit string.

[cliffords, single_qubit_list] = x.get_cliffords(r_mat_evals,sym_list)
print('Following are the qubits which are tappered off.')
print(single_qubit_list)

# exit()


print("Trying to tapering")
correct_sector = None
for taper_coeff in itertools.product([1, -1], repeat=len(single_qubit_list)):
    tapered_qubit_op = Operator.qubit_tapering(v_qubit_op, cliffords, single_qubit_list, list(taper_coeff))
    ee = ExactEigensolver(tapered_qubit_op, k=1)
    ee_result = ee.run()
    temp_min_eigvals = ee_result['eigvals'][0]
    if np.isclose(temp_min_eigvals, ref_min_eigvals, rtol=1e-8):
        correct_sector = list(taper_coeff)
    print("at sector {}: eig value: {}; reference: {}".format(list(taper_coeff), temp_min_eigvals, ref_min_eigvals.real))

# correct_sector=[1.,1.,1.,-1.,-1.]

# Get the tappered qubit operator
tapered_qubit_op = x.get_tapered_qubit_op(v_qubit_op,cliffords,single_qubit_list,correct_sector)
ee = ExactEigensolver(tapered_qubit_op.copy(), k=6)
ee_result = ee.run()
print('Getting the eigen values of the tappered off qubit operator')
print(ee_result['eigvals'][0:6])

if run_vqe==True:
    init_state = HartreeFock(num_qubits=qub_op.num_qubits- len(single_qubit_list), num_orbitals=x.fer_op.modes,
                        qubit_mapping='jordan_wigner', two_qubit_reduction=False,
                        num_particles=x.num_particles, sq_list=single_qubit_list)

    # setup variationl form
    var_form = UCCSD(num_qubits=qub_op.num_qubits- len(single_qubit_list), depth=1,
                    num_orbitals=x.fer_op.modes, 
                    num_particles=x.num_particles,
                    active_occupied=None, active_unoccupied=None, initial_state=init_state,
                    qubit_mapping='jordan_wigner', two_qubit_reduction=False, 
                    num_time_slices=1,
                    #    )
                    cliffords=cliffords, sq_list=single_qubit_list, tapering_values=correct_sector, symmetries=sym_list)

    # var_form = RYRZ(num_qubits=qub_op.num_qubits- len(single_qubit_list), depth=30,entanglement='linear')
    # print(type(var_form))
    # setup optimizer
    optimizer = COBYLA(maxiter=2000)
    # import pdb; pdb.set_trace()
    # set vqe
    algo = VQE(tapered_qubit_op, var_form, optimizer, 'matrix')

    # setup backend
    backend = BasicAer.get_backend('statevector_simulator')
    quantum_instance = QuantumInstance(backend=backend)


    algo_result = algo.run(quantum_instance)


    print(algo_result['energy'])


In [8]:
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.aqua.algorithms import ExactEigensolver
from qiskit.chemistry import QMolecule
import warnings
from qiskit.chemistry.fermionic_operator import FermionicOperator
warnings.filterwarnings("ignore",category=DeprecationWarning)
driver = PySCFDriver(atom='Be 0.000 0.0000 0.0000; H 1.291 0.0000 0.000; H -1.291,0.0000, 0.000',
                         unit=UnitsType.ANGSTROM, charge=0, spin=0, basis='sto3g')
molecule = driver.run()
one_b=QMolecule.onee_to_spin(molecule.hcore)
two_b=QMolecule.twoe_to_spin(molecule.eri)
# np.savez(MoleculeFlag+'_ao.npz',one_b=_q_.one_body_integrals,two_b=_q_.two_body_integrals)

fer_op = FermionicOperator(h1=one_b, h2=two_b)
qub_op = fer_op.mapping('jordan_wigner')
ee = ExactEigensolver(qub_op, k=6)
ee_result = ee.run()
ref_min_eigvals = ee_result['eigvals'][0:6]
print(ref_min_eigvals)


/home/k/quantum_code/test_env/lib/python3.6/site-packages/pyscf/scf/chkfile.py:31: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(chkfile) as fh5:


AttributeError: 'QMolecule' object has no attribute 'hcore'

In [5]:
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.aqua.algorithms import ExactEigensolver
from qiskit.chemistry import QMolecule
import warnings
from qiskit.chemistry.fermionic_operator import FermionicOperator
warnings.filterwarnings("ignore",category=DeprecationWarning)
driver = PySCFDriver(atom='Be 0.000 0.0000 0.0000; H 1.291 0.0000 0.000; H -1.291,0.0000, 0.000',
             unit=UnitsType.ANGSTROM,
                         charge=0, spin=0, basis='sto3g')
molecule = driver.run()
one_b = molecule.one_body_integrals
two_b = molecule.two_body_integrals
fer_op = FermionicOperator(h1=one_b, h2=two_b)
qub_op = fer_op.mapping('jordan_wigner')
ee = ExactEigensolver(qub_op, k=6)
ee_result = ee.run()
ref_min_eigvals = ee_result['eigvals'][0:6]
print(ref_min_eigvals)

/home/k/quantum_code/test_env/lib/python3.6/site-packages/pyscf/scf/chkfile.py:31: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(chkfile) as fh5:


[-19.07888937+2.81693983e-16j -18.87685668+4.28491893e-16j
 -18.87685668-4.44089210e-16j -18.87685668+3.05338463e-16j
 -18.87685668-1.21539064e-15j -18.8127128 +2.39870990e-16j]


In [9]:
mol = gto.Mole()
mol.atom = 'Be 0.000 0.0000 0.0000; H 1.291 0.0000 0.000; H -1.291,0.0000, 0.000'
mol.basis = 'sto-3g'
# mol.basis = {'O': 'sto-3g', 'H': 'cc-pvdz', 'H&#64;2': '6-31G'}

is_atomic = True
mol.build()
_q_ = qmol_func(mol, atomic=is_atomic)
two_body_temp = QMolecule.twoe_to_spin(_q_.mo_eri_ints)
temp_int = np.einsum('ijkl->ljik', _q_.mo_eri_ints)
two_body_temp = QMolecule.twoe_to_spin(temp_int)
mol = gto.M(atom=mol.atom, basis='sto-3g')

O = get_ovlp(mol)
X = np.kron(np.identity(2), np.linalg.inv(scipy.linalg.sqrtm(O)))

fer_op = FermionicOperator(h1=_q_.one_body_integrals, h2=two_body_temp)
fer_op.transform(X)
qub_op = fer_op.mapping('jordan_wigner')

ee = ExactEigensolver(qub_op, k=6)
ee_result = ee.run()
ref_min_eigvals = ee_result['eigvals'][0:6]
print(ref_min_eigvals)

converged SCF energy = -15.5613526278408
[-19.07888937+1.36871891e-16j -18.87685668-4.37761375e-17j
 -18.87685668-4.11981101e-17j -18.87685668-3.58264342e-17j
 -18.87685668+5.94099426e-17j -18.8127128 +3.07051567e-17j]
